# December 16, 2022
https://adventofcode.com/2022/day/16

In [ ]:
import re
import numpy as np

In [ ]:
class Valve:
    num_valves = 0

    def __init__( self, name, flow, neighbors ):
        self.name = name
        self.flow = flow
        self.status = "on" if self.flow == 0 else "off"
        self.neighbors = {nbr:1 for nbr in neighbors}
        self.id = Valve.num_valves
        Valve.num_valves += 1

    def is_on(self):
        return True if self.status == "on" else False

    def turn_on(self):
        if self.is_on:
            return 0
        else:
            self.status = "on"
            return self.flow

    def __lt__(self, other):
        return self.flow < other.flow
    def __eq__(self, other):
        return self.flow == other.flow
    def __le__(self, other):
        return self < other or self == other
    def __str__(self):
        return f'''Valve #{self.id} {self.name} is {self.status}. Flow = {self.flow}. Neighbors: {self.neighbors}'''

    def __repr__(self):
        return str(self)

In [ ]:
def parse_map( fn ):
    valves = []

    with open(fn, "r") as file:
        while True:
            line = file.readline().strip("\n")
            if not line: break
            ma = re.match(r'Valve (\w+).*rate=(\d+).*valves? ([\w,\s]*)', line)
            name = ma.group(1)
            flow = int(ma.group(2))
            nbrs = ma.group(3).split(", ")
            valves.append( Valve(name, flow, nbrs) )

    valves.sort(reverse=True)

    return valves


In [ ]:
test = parse_map("../data/2022/16_test.txt")

In [ ]:
print("\n".join([str(x) for x in test]))

In [ ]:
def create_graph( valves ):
    N = len(valves)
    name_map = {v.name:i for i,v in enumerate(valves)}
    dist_array = np.array( [-1]*N*N ).reshape( [N,N])
    for i in range(N):
        dist_array[i,i] = 0

    for cur_id, cur in enumerate(valves):
        dist = 0
        while True:
            # list of valves not reached yet
            tofind = [ i for i,d in enumerate( dist_array[cur_id,:] ) if d == -1 ]
            
            # end state is only the diagonals are 0
            if len(tofind) == 0:
                break
            
            # find most recent neighbors
            cur_nbrs = [ i for i,d in enumerate( dist_array[cur_id,] ) if d == dist ]
            if len(cur_nbrs) == 0:
                # uhoh--- can't get there from here!
                break

            # find neighbors of neighbors that aren't already in the distance array
            next_nbrs = set()
            for d_nbr in cur_nbrs:
                d_nbr_nbr = [ name_map[nbr] for nbr in valves[d_nbr].neighbors if name_map[nbr] in tofind ] 
                next_nbrs = next_nbrs.union( d_nbr_nbr )

            # add the new neighbor to the distance arracy
            dist += 1
            for id in next_nbrs:
                dist_array[cur_id, id] = dist
                dist_array[id, cur_id] = dist

            # end while loop for finding out cur_id distances

    # remove paths that have zero flow other than starting space
    off = [i for i,v in enumerate(valves) if v.flow==0 and v.name != "AA"]
    for i in off[::-1]:
        dist_array = np.delete(dist_array, i, 1)
        dist_array = np.delete(dist_array, i, 0)
        valves = valves[:i] + valves[i+1:]
    
    return dist_array, valves

In [ ]:
test_graph, test = create_graph(test)
test_graph

In [ ]:
test

### Part 1

Functions for memoization

In [ ]:
def get_key( start, time_left, status ):
    return f'''{start}.{time_left}.{"".join([str(s) for s in status])}'''

def memoize( score, start, time_left, status, best=None ):
    key = get_key(start, time_left, status)
    if best is None:
        best = {key:score}
    else:
        best[key] = score
    return best

def read_memo( start, time_left, status, best=None ):
    if best is None:
        return None
    key = get_key(start, time_left, status)
    if key in best:
        return best[key]
    else:
        return None

In [ ]:
def solve_map( valves, graph, start, time_left, status=None, best=None ):
    # convert to status vector to a string so we don't need to copy the whole Valve object
    if status is None:
        status = [ 1 if v.status == "on" else 0 for v in valves ]

    # read memo if possible
    best_result = read_memo( start, time_left, status, best )
    if best_result is not None:
        return best_result

    # Not previously solved. Let's solve it now!
    best_result = 0


    if time_left > 0:
        # possible places we can make it to
        # -- valve is off (s == 0)
        # -- valve is reachable and not current position (dist > 0)
        # -- valve is reachable in time (diat < time_left. We don't use <= since there must be time for an action there.)
        poss = [i for i,s in enumerate(status) if s == 0 and graph[start,i] > 0 and graph[start,i] < time_left]

        # Move to loc and turn on that valve
        for loc in poss:
            new_status = status.copy()
            new_status[loc] = 1
            new_time_left = time_left - graph[start, loc] - 1 # +1 for turning off that valve
            result = valves[loc].flow * new_time_left
            result += solve_map( valves, graph, loc, new_time_left, new_status, best )
            best_result = max(best_result, result)

    best = memoize( best_result, start, time_left, status, best=None )
    return best_result    

In [ ]:
### this is wrong now??

solve_map( test, test_graph, 0, 30 )

In [ ]:
# 16.7s without memoization
# 8.1 with memoization
puz = parse_map("data/16.txt")
puz_graph, puz = create_graph(puz)
start = [i for i,v in enumerate(puz) if v.name=="AA"][0]
print(f'''Starting at Valve #{start} ({puz[start].name}).''')
solve_map( puz, puz_graph, start, 30 )


### Part 2

In [ ]:
def get_key( start1, start2, time_left1, time_left2, status ):
    return f'''{start1}.{start2}.{time_left1}.{time_left2}.{"".join([str(s) for s in status])}'''

def memoize( score, start1, start2, time_left1, time_left2, status, best=None ): 
    key = get_key( start1, start2, time_left1, time_left2, status)

    if best is None:
        best = {key: score}
    else:
        best[key] = score
    return best

def read_memo( start1, start2, time_left1, time_left2, status, best=None ):
    key = get_key( start1, start2, time_left1, time_left2, status)

    if best is not None and key in best:
        best[key]
    else:
        return None

def get_status_cd( status ):
    return sum([ s*(10**i) for i,s in enumerate(status) ])

def get_time_left_cd( tl1, tl2 ):
    return tl1*100 + tl2

def get_start_cd(start1, start2):
    return start1*100 + start2

In [ ]:
def solve_map( valves, graph, start1, start2, time_left1, time_left2, status=None, best=None ):
    # convert to status vector to a string so we don't need to copy the whole Valve object
    if status is None:
        status = [ 1 if v.status == "on" else 0 for v in valves ]

    if time_left2 > time_left1:
        # simplify code by making id1 the person with more time left
        tmp = start2
        start2 = start1
        start1 = tmp

        tmp = time_left2
        time_left2 = time_left1
        time_left1 = tmp

    # read memo if possible
    best_result = read_memo( start1, start2, time_left1, time_left2, status, best )
    if best_result is not None:
        return best_result
    
    # Not previously solved. Let's solve it now!
    best_result = 0


    if time_left1 > 0 or time_left2 > 0:
        # possible places we can make it to
        # -- valve is off (s == 0)
        # -- valve is reachable and not current position (dist > 0)
        # -- valve is reachable in time (dist < time_left1. We don't use <= since there must be time for an action there.)
        poss = [i for i,s in enumerate(status) if s == 0 and graph[start1,i] > 0 and graph[start1,i] < time_left1]

        # edge case: person with less time is only one who can turn on an extra valve (ie. it's closer to one)
        if len(poss) == 0:
            result = solve_map(valves, graph, start1, start2, 0, time_left2, status, best)
            best_result = max(best_result, result)
        
        else:
            # Move to loc and turn on that valve
            for loc in poss:
                new_status = status.copy()
                new_status[loc] = 1
                new_time_left1 = time_left1 - graph[start1, loc] - 1 # +1 for turning off that valve
                result = valves[loc].flow * new_time_left1
                result += solve_map( valves, graph, loc, start2, new_time_left1, time_left2, new_status, best )
                best_result = max(best_result, result)
    else:
        global end_count
        end_count += 1
        if end_count % 100000 == 0:
            print(f'''Followed {end_count} chains to the end''')

    # could improve by not memoizing 0s
    best = memoize( best_result, start1, start2, time_left1, time_left2, status, best )

    return best_result    

In [ ]:
end_count = 0
solve_map( test, test_graph, 0, 0, 26, 26 )


In [ ]:
import sys
puz = parse_map("data/16.txt")
puz_graph = create_graph(puz)
start = [i for i,v in enumerate(puz) if v.name=="AA"][0]
print(f'''Starting at Valve #{start} ({puz[start].name}).''')

is_off = [i for i,v in enumerate(puz) if v.status=="off"]
Noff = len(is_off)
puz_off = np.zeros(Noff*Noff).reshape([Noff, Noff])
for i in range(Noff):
    for j in range(Noff):
        puz_off[i,j] = puz_graph[is_off[i], is_off[j] ]

# Looking for some meaningful structure to help eliminate paths
# Nothing super obvious. Maybe the middle valves tend to be more centrally located
np.set_printoptions(threshold=sys.maxsize)
print(puz_off)

In [ ]:
# 61m --- 11714002 end points (3 with tl==26)
# + 624m --- 
end_count = 0
#best = {}
solve_map( puz, puz_graph, start, start, 26, 26, best=best )

In [ ]:
len(best.keys())

In [ ]:
tl = [int(k.split(".")[2]) for k in best.keys()]

In [ ]:
sum([1 for x in tl if x ==26])

In [ ]:
best

### Part 2 -- Again
... new strat... try finding the lowest-cost path to the biggest valve
but that didn't turn out to be very helpful

Thoughts:
1. If you're going to turn on a valve, the best time to do it is when you're already there.
2. If you will turn on a valve, do it the first time you visit. The cost is the same, but the benefit is greater.
  * Turns out this is FALSE. There is extra cost in the form of delaying turning on other valves, possibly HUGE ones.
3. 



In [ ]:
test

In [ ]:
test_graph

In [ ]:
def find_biggest_valve( valves ):
    best_val = 0
    for i, valve in enumerate(valves):
        if valve.flow > best_val:
            best_i = i
            best_val = valve.flow

    return best_i

def midpoint_cost( valves, graph, time_left, start, mid, end ):
    # additional cost of going to start to end via mid and turning on that valve
    # indirect time includes one unit to turn on mid valve
    indirect_time = min(time_left, graph[start][mid] + graph[mid][end] + 1 )
    direct_time = min(time_left, graph[start][end])

    # we miss out on some flow from the end valve
    extra_cost = (indirect_time - direct_time) * valves[end].flow

    # but get additional flow from the mid valve
    bonus_flow_time = max(0, indirect_time - (graph[start][mid] + 1))
    bonus_flow = bonus_flow_time * valves[mid].flow
    return extra_cost - bonus_flow, indirect_time - direct_time

def midpoint_flow( valves, graph, time_left, start, mid, end ):
    # extra flow achieved by going start -> mid -> end and turning both valves on

    # Time for mid valve and end valve to flow
    mid_flow_time = max(time_left - graph[start][mid] - 1, 0)
    end_flow_time = max(time_left - graph[start][mid] - 1 - graph[mid][end] - 1, 0)
    # Flow time for end valve if go straight there
    direct_flow_time = max(time_left - graph[start][end] - 1, 0)
    # Extra time to stop and turn on mid
    lost_time = end_flow_time - direct_flow_time 
    # Extra flow is the flow from mid - the lost flow from end for taking extra time to get there
    extra_flow = mid_flow_time * valves[mid].flow - lost_time * valves[end].flow 
    
    return extra_flow, lost_time

In [ ]:
time_left = 26
valves = test
graph = test_graph
start = 0
end = find_biggest_valve( valves )
# every minute of delay reduces flow by this amount

status = [0] * len(valves)
best_cost = 0
best_i = end

def paths_to_try( valves, graph, time_left, status, cur, end ):
    # Get all valid paths from cur to end
    # A path is valid if it's direct or it produces more flow
    # More flow is not necessarily better since it costs more time
    # But direct is better than any path with less flow
    # Note: We are not comparing against paths that do attempt to reach end

    # List of valid paths so far (excluding end)
    paths = [ [start] ] 

    # amount of flow generated by using this path for duration of time_left
    flows = [ (time_left - graph[start][end] - 1)*valves[end].flow ]
    
    # time for paths from start, but excluding end
    times = [0]

    id = 0
    while id < len(paths):
        cur = paths[id]

        # see if we could add another step to the path
        for i in range(len(valves)):
            # this valve is already on path, or else the valve is off
            if i == start or i == end or i in cur or status[i] == 1:
                continue

            # consider stopping at i en route to end.
            # add to paths if it's possibly better than going from current path straight to end
            extra_flow, lost_time = midpoint_flow( valves, graph, time_left - times[id], cur[-1], i, end)
            if extra_flow > 0:
                print( cur, i, extra_flow)
                paths.append( cur + [i] )
                flows.append( flows[id] + extra_flow )
                times.append( times[id] + lost_time )
        id += 1
    return paths, 

best_cost = 1
best_i = []
for i, cost in enumerate(costs):
    if cost < best_cost:
        best_i = [i]
        best_cost = cost
    elif cost == best_cost and set():
        best_i.append(i)




In [ ]:
set([1,4,3]) == set([3,4,2])

In [ ]:
paths

In [ ]:
costs

In [ ]:
costs

In [ ]:
[
    6*13 - 3*22,
    4*20 - 22,
    7*21 - 5*22,
    7*13 + 4*20 - 4*22,
    11*13 + 7*21 - 8*22,
    10*21 + 6*13 - 8*22,
    
]

In [ ]:
test_graph

In [ ]:
midpoint_cost( valves, graph, 32, 0, 6, 5)

In [ ]:
(3+2+4+2)*21 + (2+4+1)*13 + 4*20 - (14-5)*22

In [ ]:
midpoint_cost(valves, graph, 32, 0, 1, 5)

In [ ]:
test_graph

In [ ]:
(3+3+4)*13 + (7*21) + 4*20 - 9*22

In [ ]:
test

### Part 2 -- Third attempt
Let's try an A* type deal and avoid exploring paths that we know won't win

# Too low 2134, 2493, 2714
# Too high 3000

In [ ]:
from queue import PriorityQueue

In [ ]:
class Strat:
    next_id = 0

    def __init__(self, phum, pele, thum, tele, score, prio=0, tarrh=[], tarre=[] ):
        self.id = Strat.next_id
        Strat.next_id += 1

        # paths for human and elephant
        self.phum = phum
        self.pele = pele

        # time left for them
        self.thum = thum
        self.tele = tele

        self.score = score
        self.prio = prio

        # These are arrays of the time_left values at each point along path
        # Not necessary, but it helped me debug
        self.tarrh = tarrh
        self.tarre = tarre

        return

    # Note: for our purpose < means has higher priority
    def __lt__(self, other):
        return self.prio > other.prio

    def __eq__(self, other):
        # Must have same score
        if self.score != other.score:
            return False

        # With actors in same position at same time, possibly swapped though
        if not (
            (self.phum[-1] == other.phum[-1] and self.thum == other.thum
                and self.pele[-1] == other.pele[-1] and self.tele == other.tele)
            or 
            (self.phum[-1] == other.pele[-1] and self.thum == other.tele
                and self.pele[-1] == other.phum[-1] and self.tele == other.thum)
        ):
            return False

        # and the same valves must be on
        on1 = set( self.phum + self.pele )
        on2 = set( other.phum + other.pele )
        if on1 != on2:
            return False
        
        # in that case, previous history is immaterial
        return True



In [ ]:
def get_nbr( strat, graph ):
    '''return valves we might check'''
    # We assume starting valve has flow 0
    return [i for i in range(graph.shape[0]) if i not in strat.phum and i not in strat.pele]


In [ ]:
def calc_time_left( path, cur_time, nbr, graph ):
    cur = path[-1]
    dist = graph[cur][nbr]

    # time left after reaching valve and turning it on, or 0 if you can't make it
    return max(0, cur_time - dist - 1)

def heuristic1( strat, valves, graph ):
    '''An upper bound assuming all valves are 1 unit away from each other'''
    # Assume valves are sorted
    valves_left = get_nbr( strat, graph )
    flows = [valves[id].flow for id in valves_left]
    
    # Upper bound assumes all valves are 1 unit away from each other
    # we can improve on this upper bound if needed
    ub = 0
    t1 = strat.thum
    t2 = strat.tele
    for f in flows:
        if t1 < t2 and t2 > 2:
            ub += (t2-2)*f
            t2 -= 2
        elif t1 > 2:
            ub += (t1-2)*f
            t1 -= 2
        else:
            # no more time to turn stuff on
            break

    return ub

def heuristic( strat, valves, graph ):
    '''an upper bound assuming that you and ele can simultaneously go toward all remaining valves'''
    # Assume valves are sorted
    valves_left = get_nbr( strat, graph )
    flows = [valves[id].flow for id in valves_left]
    
    curh = strat.phum[-1]
    cure = strat.pele[-1]
    tlh = strat.thum
    tle = strat.tele

    flow = [ valves[v].flow * max(0, tlh - graph[curh][v], tle - graph[cure][v]) for v in valves_left ]
    return sum(flow) 




In [ ]:
import datetime as dt

In [ ]:
def this_could_be_it( valves, graph, start, time ):
    
    # Heuristic will be max possible flow
    # Priority will be negated to sort from high to low 
    frontier = PriorityQueue()

    #   hum path, ele path, hum time, ele time, score
    start = Strat( [start], [start], time, time, 0, 0, [time], [time] )
    frontier.put( start )

    # track which ones we've already investigated
    seen = [start]
    best_strat = start
    best_score = -1
    nbest = 1
    
    while not frontier.empty():
        cur = frontier.get()

        # this strat can't possibly win, skip it
        # remember prio is negative of the upper bound on this strats final score
        if cur.prio <= best_score:
            continue

        neighbors = get_nbr(cur, graph)
        for nbr in neighbors:
            
            h_time_left = calc_time_left( cur.phum, cur.thum, nbr, graph )
            e_time_left = calc_time_left( cur.pele, cur.tele, nbr, graph )

            # I'm not 100% sure this works, but it will lead to heuristics that aren't so crazy inflated.
            # This led quickly to a wrong answer of 2714 in ~43s

            # On second thought this makes sense. Even if human ultimately goes to nbr,
            # We can move the ele to its next destination first...
            # Eventually ele will use up its extra time
            # Maybe the bug earlier was because I had a strict inequality here?
            
            try_hum = h_time_left >= e_time_left
            try_ele = e_time_left >= h_time_left and (cur.pele[-1] != cur.phum[-1] or cur.tele != cur.thum)

            # better condition, if ele has time to turn on valve and return to spot, then no point in human doing it
            # WHOOPS... the problem with this is it forces the elephant away from a possibly more efficient position... maybe?
            # not sure, but I got the wrong answer of 2714 again
            #try_hum = (h_time_left > e_time_left - graph[nbr][cur.pele[-1]])
            #try_ele = (e_time_left > h_time_left - graph[nbr][cur.phum[-1]]
            #            and (cur.pele[-1] != cur.phum[-1] or cur.tele != cur.thum))

            # This MUST be okay... if both human and ele are in same spot at same time, it doesn't matter who moves
            #try_hum = True
            #try_ele = (cur.pele[-1] != cur.phum[-1] or cur.pele[-1] != cur.phum[-1])

            # Try Human...
            if try_hum:
                time_left = h_time_left
                new_score = cur.score + time_left * valves[nbr].flow # time_left = time when valve is opened, it starts flowing next minute

                new_strat = Strat( [*cur.phum, nbr], cur.pele, time_left, cur.tele, new_score, 0, [*cur.tarrh, time_left], cur.tarre )
                if True:#new_strat not in seen:
                    if new_score > best_score:
                        nbest += 1
                        print(f"Best [{nbest}]: {new_score}")
                        print(dt.datetime.now().isoformat())
                        best_score = new_score
                        best_strat = new_strat
                    heur = heuristic( new_strat, valves, graph )
                    prio = new_score + heur
                    new_strat.prio = prio
                    #seen.append( new_strat )

                # don't bother queuing this new strat unless it could top the best score
                # Note: if this is the current best_score, it means we only add if there's time to turn on more valves
                if prio > best_score:
                    frontier.put( new_strat )

            # Try Elephant...
            if try_ele:
                time_left = e_time_left
                new_score = cur.score + time_left * valves[nbr].flow # time_left = time for this valve to flow
                new_strat = Strat( cur.phum, [*cur.pele, nbr], cur.thum, time_left, new_score, 0, cur.tarrh, [*cur.tarre, time_left] )

                if True:#new_strat not in seen:
                    if new_score > best_score:
                        nbest += 1
                        print(f"Best [{nbest}]: {new_score}")
                        print(dt.datetime.now().isoformat())
                        best_score = new_score
                        best_strat = new_strat
                    heur = heuristic( new_strat, valves, graph )
                    prio = new_score + heur
                    new_strat.prio = prio
                    #seen.append(new_strat)

                # don't bother queuing this new strat unless it could top the best score
                # Note: if this is the current best_score, it means we only add if there's time to turn on more valves
                if prio > best_score:
                    frontier.put( new_strat )

    # no more strats to check out
    return best_score, best_strat

            



        



In [ ]:
start = [i for i,v in enumerate(test) if v.name == "AA"][0]
score, strat = this_could_be_it( test, test_graph, start, 26)
print(score)

In [ ]:
print([test[x].name for x in strat.phum])
print([test[x].name for x in strat.pele])
print(strat.thum)
print(strat.tele)

In [ ]:
def score_strat( strat, valves, graph, time=26, hum=1 ):
    if hum == 1:
        n1 = "You turn"
        n2 = "Ele turns"
    else:
        n1 = "Ele turns"
        n2 = "You turn"
    
    tl1 = tl2 = time
    i1 = i2 = 0
    fpt = 0
    total_flow = 0
    while tl1 > 0 or tl2 > 0:
        print(i1, i2)
        if i1 < len(strat.phum) - 1:
            tl1b = tl1 - (graph[ strat.phum[i1] ][ strat.phum[i1+1] ] + 1)
        else:
            tl1b = 0
        if i2 < len(strat.pele) - 1:
            tl2b = tl2 - (graph[ strat.pele[i2] ][ strat.pele[i2+1] ] + 1)
        else:
            tl2b = 0
        if tl1b == tl2b == 0:
            break
        
        if tl1b >= tl2b:
            i1 += 1
            tl1 = tl1b
            v = valves[ strat.phum[i1] ]
            fpt += v.flow
            total_flow += (tl1)*v.flow
            print(f"@{time-tl1} {n1} on Valve {v.name} releasing {v.flow} and increasing FPT to {fpt}.")
            print("Total =", total_flow)
        else:
            i2 += 1
            tl2 = tl2b
            v = valves[ strat.pele[i2] ]
            fpt += v.flow
            total_flow += (tl2)*v.flow
            print(f"@{time-tl2} {n2} on Valve {v.name} releasing {v.flow} and increasing FPT to {fpt}.")
            print("Total =", total_flow)

    return total_flow


    

In [ ]:
score_strat( strat, test, test_graph, time=26, hum=1 )


In [ ]:
def print_strat( strat, valves ):
    print( "One:",  [f"{valves[ strat.phum[i] ].name} @{26 - strat.tarrh[i]}" for i in range(len(strat.phum))])
    print( "Two:",[f"{valves[ strat.pele[i] ].name} @{26 - strat.tarre[i]}" for i in range(len(strat.pele))])

In [ ]:
print_strat(strat, test)

In [ ]:
print([f"{v.name}: {v.flow}" for v in test])
test_graph

In [ ]:
puz = parse_map("data/16.txt")
puz_graph, puz = create_graph(puz)

In [ ]:
# The h_time_left <= e_time_left shortcut, which I now believe is good but with prio shortcut turned off
# LMFAO! I had the wrong starting tile this entire time.... *sigh*
start = len(puz)-1
score, strat = this_could_be_it( puz, puz_graph, start, 26)
print(score)

In [ ]:
# The h_time_left <= e_time_left shortcut, which I now believe is good
# gets 2714 in ~43s
score, strat = this_could_be_it( puz, puz_graph, start, 26)
print(score)

In [ ]:
# The only shortcut here is if human and ele are in same spot at same time, we only try moving one of them
# it gives wrong answer of 2714 in 13'48"
score, strat = this_could_be_it( puz, puz_graph, start, 26)
print(score)

In [ ]:
# This one had condition where if ele can turn on valve and return to its spot, then no need for human to move
# it gives the wrong answer of 2714 in 13'22"
score, strat = this_could_be_it( puz, puz_graph, start, 26)
print(score)

In [ ]:
score, strat = this_could_be_it( puz, puz_graph, start, 26)
print(score)

In [ ]:
# I forget what I added here, but it took too long anyway
score, strat = this_could_be_it( puz, puz_graph, start, 26)
print(score)

In [ ]:
print_strat(strat, puz)

In [ ]:
strat.phum, strat.tarrh, strat.pele, strat.pele

In [ ]:
get_nbr(strat,puz)

In [ ]:
strat

In [ ]:
print([f"{v.name}: {v.flow}" for v in puz])
puz_graph

In [ ]:
#long_strat = strat
#long_score = score
print_strat(long_strat, puz)

In [ ]:
#fast_strat = strat
#fast_score = score
print_strat(fast_strat, puz)

In [ ]:
score_strat( fast_strat, puz, puz_graph )

In [ ]:
strat.phum, strat.thum, strat.tarrh

In [ ]:
puz_graph